# Visual Analytics

## Assignment 3

**Instructor:** Dr. Marco D'Ambros  
**TAs:** Carmen Armenti, Mattia Giannaccari

**Contacts:** marco.dambros@usi.ch, carmen.armenti@usi.ch, mattia.giannaccari@usi.ch

**Due Date:** May 16, 2025 @ 23:55

---
The goal of this assignment is to use **Spark (PySpark)** and **Polars** in Jupyter notebooks.  
The files `trip_data.csv`, `trip_fare.csv`, and `nyc_boroughs.geojson` are available in the provided folder: [Assignment3-data](https://usi365-my.sharepoint.com/:f:/g/personal/armenc_usi_ch/Ejp7sb8QAMROoWe0XUDcAkMBoqUFk-w2Vgroup025NhAww?e=2I7SMC).

You may clean the data as needed; however, please note that specific data cleaning steps will be required in **Exercise 5**. If you choose to clean the data before Exercise 5, make sure to retain the **original dataset** for use with the Polars exercises.

- Use **Spark** to solve **Exercises 1–4**
- Use **Polars** to solve **Exercises 5–8**

You are encouraged to use [Spark window functions](https://spark.apache.org/docs/latest/sql-ref-syntax-qry-select-window.html) whenever appropriate.

Please name your notebook file as `SurnameName_Assignment3.ipynb`

In [1]:
%pip install pyspark
%pip install polars
%pip install bokeh
%pip install geopandas
%pip install bokeh_sampledata

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Spark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date

### Exercise 1
Join the `trip_data` and `trip_fare` dataframes into one and consider only data on 2013-01-01. Please specify the number of rows obtained after joining the 2 datasets.

The first thing to do is to create a `SparkSession`

In [3]:
spark = SparkSession.builder.getOrCreate()

spark.conf.set("spark.sql.shuffle.partitions", 400)
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

25/05/06 15:59:43 WARN Utils: Your hostname, USILU-16210.local resolves to a loopback address: 127.0.0.1; using 192.168.43.129 instead (on interface en0)
25/05/06 15:59:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/06 15:59:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
trip_data_df = spark.read.csv('./data/trip_data.csv', header=True, inferSchema=True)
trip_fare_df = spark.read.csv('./data/trip_fare.csv', header=True, inferSchema=True)

Before joining the 2 dataframe I am going to filter the datas by the given data.

In [5]:
trip_data_df = trip_data_df.select([col(c).alias(c.strip()) for c in trip_data_df.columns])
trip_data_df.printSchema()

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- rate_code: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_time_in_secs: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)



In [6]:
trip_fare_df = trip_fare_df.select([col(c).alias(c.strip()) for c in trip_fare_df.columns])
trip_fare_df.printSchema()

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- surcharge: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)



Since we have 2 temporal metric I am going to filter on both in order to avoid `pickup_datetime` being 2012-12-31 while `dropoff_datetime` is 2013-01-01 and in similar way I want to avoid `pickup_datetime` being 2013-01-01 while `dropoff_datetime` is 2013-01-02

In [7]:
trip_data_filtered_df = trip_data_df.filter(
    (to_date("pickup_datetime") == "2013-01-01") &
    (to_date("dropoff_datetime") == "2013-01-01")
)

trip_fare_filtered_df = trip_fare_df.filter(
    (to_date("pickup_datetime") == "2013-01-01")
)

I can now merge the 2 dataframes using the `join` per `medallion`, `hack_license`, `vendor_id` and `pickup_datetime`

In [8]:
trip_filtered_df = trip_data_filtered_df.join(
    trip_fare_filtered_df,
    on=["medallion", "hack_license", "vendor_id", "pickup_datetime"],
    how="inner"
)

In [9]:
print(f"The number of rows are: {trip_filtered_df.count()}")

The number of rows are: 410816


### Exercise 2
Provide a graphical representation to compare the average fare amount for trips _within_ and _across_ all the boroughs. You may want to have a look at: https://docs.bokeh.org/en/latest/docs/user_guide/topics/categorical.html#categorical-heatmaps

In [10]:
import geopandas as gpd

from pyspark.sql.functions import max as spark_max, min as spark_min

from bokeh.plotting import figure, show, reset_output, output_notebook
from bokeh.models import GeoJSONDataSource

reset_output()
output_notebook()

Loading BokehJS ...

In [11]:
borough_gdf = gpd.read_file('./data/nyc-boroughs.geojson')

In [12]:
trip_filtered_pd_df = trip_filtered_df.toPandas()

pickup_gdf = gpd.GeoDataFrame(
    trip_filtered_pd_df,
    geometry=gpd.points_from_xy(trip_filtered_pd_df['pickup_longitude'], trip_filtered_pd_df['pickup_latitude']),
    crs=borough_gdf.crs
)

dropoff_gdf = gpd.GeoDataFrame(
    trip_filtered_pd_df,
    geometry=gpd.points_from_xy(trip_filtered_pd_df['dropoff_longitude'], trip_filtered_pd_df['dropoff_latitude']),
    crs=borough_gdf.crs
)

pickup_with_borough = gpd.sjoin(pickup_gdf, borough_gdf[['borough', 'geometry']], how='left', predicate='within')
dropoff_with_borough = gpd.sjoin(dropoff_gdf, borough_gdf[['borough', 'geometry']], how='left', predicate='within')

trip_filtered_pd_df['pickup_borough'] = pickup_with_borough['borough']
trip_filtered_pd_df['dropoff_borough'] = dropoff_with_borough['borough']

trip_filtered_df = spark.createDataFrame(trip_filtered_pd_df)

In [13]:
from bokeh.models import BasicTicker, PrintfTickFormatter
from bokeh.plotting import figure, show
from bokeh.transform import linear_cmap

trip_group_df = trip_filtered_df \
    .groupBy(['pickup_borough', 'dropoff_borough']) \
    .avg('fare_amount') \
    .withColumnRenamed('avg(fare_amount)', 'avg_fare')

pickup_borough = borough_gdf['borough'].unique()
dropoff_borough = borough_gdf['borough'].unique()

min = trip_group_df.agg(spark_min('avg_fare')).collect()[0][0]
max = trip_group_df.agg(spark_max('avg_fare')).collect()[0][0]

colors = ["#03045e", "#023e8a", "#0077b6", "#0096c7", "#00b4d8", "#48cae4", "#90e0ef", "#ade8f4", "#caf0f8"]

TOOLS = "hover"
TOOLTIPS = [
    ('Pickup Borough', '@pickup_borough'),
    ('Dropoff Borough', '@dropoff_borough'),
    ('Average Fare Amount', '@avg_fare{0.2f}')
]

p = figure(title="Average Fare Amount for Pickup and Dropoff Boroughs",
           x_range=pickup_borough, y_range=dropoff_borough,
           x_axis_location="above", width=900, height=400,
           tools=TOOLS, toolbar_location='below', tooltips=TOOLTIPS)

p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "7px"
p.axis.major_label_standoff = 0

r = p.rect(x="pickup_borough", y="dropoff_borough", width=1, height=1, source=trip_group_df.toPandas(),
           fill_color=linear_cmap("avg_fare", colors[::-1], low=min, high=max),
           line_color=None)

p.add_layout(r.construct_color_bar(
    major_label_text_font_size="7px",
    ticker=BasicTicker(desired_num_ticks=len(colors)),
    formatter=PrintfTickFormatter(format="%d%%"),
    label_standoff=6,
    border_line_color=None,
    padding=5,
), 'right')

show(p)

25/05/06 16:00:58 WARN TaskSetManager: Stage 18 contains a task of very large size (8600 KiB). The maximum recommended task size is 1000 KiB.
25/05/06 16:00:59 WARN TaskSetManager: Stage 24 contains a task of very large size (8600 KiB). The maximum recommended task size is 1000 KiB.
25/05/06 16:01:00 WARN TaskSetManager: Stage 30 contains a task of very large size (8600 KiB). The maximum recommended task size is 1000 KiB.


### Exercise 3
Consider only Manhattan, Bronx and Brooklyn boroughs. Then create a dataframe that shows the total number of trips *within* the same borough and *across* all the other boroughs mentioned before (Manhattan, Bronx, and Brooklyn) where the passengers are more or equal than 3.

For example, for Manhattan borough you should consider the total number of the following trips:
- Manhattan → Manhattan
- Manhattan → Bronx
- Manhattan → Brooklyn

You should then do the same for Bronx and Brooklyn boroughs.

In [15]:
trip_filtered_df.printSchema()

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- rate_code: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_time_in_secs: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- surcharge: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- pickup_borough: string (nullable = true)
 |-- dr

In [20]:
boroughs = ['Manhattan', 'Brooklyn', 'Bronx']
min_passenger = 3

trip_group_bp_df = trip_filtered_df \
    .filter(
        (col('pickup_borough').isin(boroughs)) &
        (col('dropoff_borough').isin(boroughs)) &
        (col('passenger_count') >= min_passenger)
    ) \
    .groupBy(['pickup_borough', 'dropoff_borough']) \
    .count() \
    .withColumnRenamed('count', 'trip_count') \
    .orderBy('trip_count', ascending=False)

trip_group_bp_df

25/05/06 16:06:53 WARN TaskSetManager: Stage 58 contains a task of very large size (8600 KiB). The maximum recommended task size is 1000 KiB.
25/05/06 16:06:54 WARN TaskSetManager: Stage 61 contains a task of very large size (8600 KiB). The maximum recommended task size is 1000 KiB.


pickup_borough,dropoff_borough,trip_count
Manhattan,Manhattan,62391
Manhattan,Brooklyn,2762
Brooklyn,Brooklyn,2014
Brooklyn,Manhattan,1326
Manhattan,Bronx,527
Bronx,Bronx,103
Bronx,Manhattan,55
Brooklyn,Bronx,10
Bronx,Brooklyn,2


### Exercise 4
Create a dataframe where each row represents a driver, and there is one column per borough.
For each driver-borough, the dataframe provides the maximum number of consecutive trips
for the given driver, within the given borough. Please consider only trips which were payed by card. 

For example, if for driver A we have (sorted by time):
- Trip 1: Bronx → Bronx
- Trip 2: Bronx → Bronx
- Trip 3: Bronx → Manhattan
- Trip 4: Manhattan → Bronx.
    
The maximum number of consecutive trips for Bronx is 2.

## Polars

### Exercise 5

Please work on the merged dataset of trips and fares and perform the following data cleaning tasks:

1. Remove trips with invalid locations (i.e. not in New York City);
3. Remove trips with invalid amounts:
    - Total amount must be greater than zero;
    - Total amount must correspond to the sum of all the other amounts.
5. Remove trips with invalid time:
    - Pick-up before drop-off;
    - Valid duration.

After each data cleaning task, report how many rows where removed. Finally report:
- Are there **duplicate trips**?
- How many trips remain after cleaning?

In [14]:
import polars as pl
import chardet

ModuleNotFoundError: No module named 'chardet'

In [ ]:
with open('data/trip_fare.csv', 'rb') as f:
    data = f.read()

encoding_result = chardet.detect(data)
encoding = encoding_result['encoding']
print(f"Detected encoding: {encoding}")

Detected encoding: ascii


In [ ]:
tripdata_polar_df = pl.read_csv('./data/trip_data.csv')
tripfare_polar_df = pl.read_csv('./data/trip_fare.csv', encoding=encoding)


ComputeError: could not parse `1.75` as dtype `i64` at column ' tip_amount' (column number 9)

The current offset in the file is 17600 bytes.

You might want to try:
- increasing `infer_schema_length` (e.g. `infer_schema_length=10000`),
- specifying correct dtype with the `schema_overrides` argument
- setting `ignore_errors` to `True`,
- adding `1.75` to the `null_values` list.

Original error: ```remaining bytes non-empty```

### Exercise 6

Compute the **total revenue** (total_amount) grouped by:
- Pick-up hour of the day (0–23)
- Passenger count (group >=6 into “6+”)

Create a heatmap where:
- X-axis = hour
- Y-axis = passenger count group
- Cell value = average revenue per trip

### Exercise 7

Define an "anomalous trip" as one that satisfies at least two of the following:
- Fare per mile is above the 95th percentile
- Tip amount > 100% of fare
- trip_time_in_secs is less than 60 seconds but distance is more than 1 mile

Create a dataframe of anomalous trips and:
- Report how many such trips exist
- Create a scatterplot to visualize the anomaly metrics
- Describe the visualization identifying groups and outliers

### Exercise 8
For each driver (hack_license), calculate the **total profit per hour worked**, where:
> profit = 0.7 * (fare_amount + tip_amount) when the trip starts between 7:01 AM and 7:00 PM\
> profit = 0.8 * (fare_amount + tip_amount) when the trip starts between 7:01PM and 7:00 AM

Estimate "hours worked" by summing trip_time_in_secs.

Plot a line chart showing the distribution of average profit per hour **for the top 10% drivers** in terms of total trips.

Which time of day offers **best earning efficiency**?